## Support Vector Machine for tagging GOV.UK

### Load requirements and data

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.svm import LinearSVC
from sklearn.decomposition import TruncatedSVD

/Users/matthewupson/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Double check we are in the right environment!
!which python3
!python3 --version

/Users/matthewupson/.pyenv/shims/python3
Python 3.4.6


In [5]:
# Becuase the file is large it uses the low_memory option 
# to read the file in chunks. The result of this is that it
# it infers different column types in the different chunks,
# solve this either by reading the file in a oner with low_memory=False,
# or be explicit about the column types by setting them with dtype = {'column':type}

labelled_level2 = pd.read_csv(
    '../../../data/labelled_level2.csv',
    low_memory=False
)

In [6]:
labelled_level2.shape

(173560, 21)

In [7]:
# Collapse down World

labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

In [8]:
# There are are 210 unique taxons remaining in level2taxon

labelled_level2['level2taxon'].nunique()

210

In [9]:
# Drop cases where there are multiple taxons applied to a content item.
# This is the simplest solution to the multiple class problem

print('Before:', labelled_level2.shape)
labelled_level2.drop_duplicates(subset=['content_id'], inplace=True, keep='first')
print('After: ', labelled_level2.shape)

Before: (173560, 21)
After:  (114048, 21)


In [10]:
document_type_encoder = LabelEncoder()
publishing_app = LabelEncoder()
document_type_gp_encoder = LabelEncoder()
locale_encoder = LabelEncoder()

labelled_level2['document_type'] = document_type_encoder.fit_transform(labelled_level2['document_type'])
labelled_level2['publishing_app'] = document_type_encoder.fit_transform(labelled_level2['publishing_app'])
labelled_level2['document_type_gp'] = document_type_encoder.fit_transform(labelled_level2['document_type_gp'])
labelled_level2['locale'] = document_type_encoder.fit_transform(labelled_level2['locale'])


Note that when combining the metadata features with the text features, the text features are a very large sparse matrix (>100000, >100000) whilst the metadata features are (>100000, 4). Attempted to shrink the text matric with LSA to 100 features, however this is repeatedly failing with memory allocation errors.

In [13]:
comment_pipeline = Pipeline([
                   ('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer(sublinear_tf=True, use_idf=True)),
                   ('perc', SelectPercentile(score_func=f_classif, percentile=2)),
])
    
    
    
    #LinearSVC(C=10.0, dual=False, loss="squared_hinge", penalty="l2", tol=0.01)

This feature selection method reduce the dimensions to c.3000 from 100,000

In [15]:
text = comment_pipeline.fit_transform(X=labelled_level2['combined_text'],y=labelled_level2['level2taxon'])

/Users/matthewupson/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [24]:
text = text.todense()

(114048, 3370)

In [25]:
metadata = labelled_level2.filter(
    ['document_type', 'primary_publishing organisation', 
     'publishing_app', 
     'document_type_gp', 'locale'], 
    axis=1)

In [27]:
df = pd.concat([text,metadata], axis=1)

TypeError: cannot concatenate object of type "<class 'numpy.matrixlib.defmatrix.matrix'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    labelled_level2['combined_text'], labelled_level2['level2taxon'], test_size = 0.2, random_state=1337)

In [ ]:
svc_clf = Pipeline([
                   ('svc', LinearSVC(C=10.0, dual=False, loss="squared_hinge", penalty="l2", tol=0.01))
                   ])

In [ ]:
svc_clf.fit(X_train, y_train)

In [ ]:
""" Predict the test dataset using SVM"""
predicted_svm = svc_clf.predict(X_test)
print('SVM correct prediction: {:4.2f}'.format(np.mean(predicted_svm == y_test)))

In [ ]:
print(classification_report(y_test, predicted_svm))


## Metadata classifier

In [ ]:
labelled_level2.columns

In [ ]:
metadata = labelled_level2.filter(['document_type', 'primary_publishing organisation', 'publishing_app', 'document_type_gp', 'locale'], axis=1)

In [ ]:
metadata.columns

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(
    metadata, labelled_level2['level2taxon'], test_size = 0.2, random_state=1337)

In [ ]:
ada_clf = AdaBoostClassifier()

ada_clf.fit(X_train, y_train)

In [ ]:
predicted_ada = ada_clf.predict(X_test)

In [ ]:
print('Adaboost correct prediction: {:4.2f}'.format(np.mean(predicted_ada == y_test)))

In [ ]:
print(classification_report(y_test, predicted_ada))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfo_clf = RandomForestClassifier()
rfo_clf.fit(X_train, y_train)

In [ ]:
predicted_rfo = rfo_clf.predict(X_test)
print('Adaboost correct prediction: {:4.2f}'.format(np.mean(predicted_rfo == y_test)))

In [ ]:
print(classification_report(y_test, predicted_ada))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
predicted_rfo = rfo_clf.predict_proba(X_test)
predicted_ada = ada_clf.predict_proba(X_test)

In [ ]:
blended = pd.DataFrame(np.c_[
    predicted_rfo, predicted_ada
])

In [ ]:
blended_model = LogisticRegression(random_state=1337)
blended_model.fit(blended, y_test)

In [ ]:
blended_pred = blended_model.predict(blended)
class_report = classification_report(y_test, blended_pred)
print(class_report)
blended_model.get_params()

In [ ]:
# Double check this with CV

scores = cross_val_score(blended_model, X_test, y_test,
                        scoring='accuracy',cv=10)

print(round(scores.mean(),3))
print(round(scores.std(),3))
#72